In [1]:
# Utilities
import matplotlib.pyplot as plt
import pandas as pd
import getpass
import requests

from sentinelhub import (
    SHConfig,
    DataCollection,
    SentinelHubCatalog,
    SentinelHubRequest,
    SentinelHubStatistical,
    BBox,
    bbox_to_dimensions,
    CRS,
    MimeType,
    Geometry,
)

from utils import plot_image

In [2]:
config = SHConfig()
config.sh_client_id = getpass.getpass("sh-4508d980-b7ed-44f0-845d-95b368b9869c")
config.sh_client_secret = getpass.getpass("uGDI7nYPrd5uH0U8rXEHFbDfy61Xeswi")
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"

sh-4508d980-b7ed-44f0-845d-95b368b9869c ········
uGDI7nYPrd5uH0U8rXEHFbDfy61Xeswi ········


In [3]:
# define functions to extract statistics for all acquisition dates
def extract_stats(date, stat_data):
    d = {}
    for key, value in stat_data["outputs"].items():
        stats = value["bands"]["B0"]["stats"]
        if stats["sampleCount"] == stats["noDataCount"]:
            continue
        else:
            d["date"] = [date]
            for stat_name, stat_value in stats.items():
                if stat_name == "sampleCount" or stat_name == "noDataCount":
                    continue
                else:
                    d[f"{key}_{stat_name}"] = [stat_value]
    return pd.DataFrame(d)


def read_acquisitions_stats(stat_data):
    df_li = []
    for aq in stat_data:
        date = aq["interval"]["from"][:10]
        df_li.append(extract_stats(date, aq))
    return pd.concat(df_li)

In [4]:
aoi_coords_wgs84 = [15.56, 46.84, 15.574922, 46.851514]

In [5]:
def get_open_elevation(lat, lon):
    query = f'https://api.open-elevation.com/api/v1/lookup?locations={lat},{lon}'
    response = requests.get(query).json()
    # Check if the response contains results
    if 'results' in response:
        return response['results'][0]['elevation']
    else:
        return None

# Calculate the average elevation
elevations = [
    get_open_elevation(aoi_coords_wgs84[1], aoi_coords_wgs84[0]),
    get_open_elevation(aoi_coords_wgs84[3], aoi_coords_wgs84[0]),
    get_open_elevation(aoi_coords_wgs84[1], aoi_coords_wgs84[2]),
    get_open_elevation(aoi_coords_wgs84[3], aoi_coords_wgs84[2]),
]

# Filter out None values in case some API calls failed
elevations = [elev for elev in elevations if elev is not None]

if elevations:
    average_elevation = sum(elevations) / len(elevations)
    print(f"The average elevation is {average_elevation} meters.")
else:
    print("Could not retrieve elevation data.")

The average elevation is 281.5 meters.


In [6]:
catalog = SentinelHubCatalog(config=config)

In [7]:
evalscript_ndvi = """
//VERSION=3
function setup() {
  return {
    input: [{
      bands: [
        "B04",
        "B08",
        "dataMask"
      ]
    }],
    output: [
      {
        id: "ndvi",
        bands: 1
      },
      {
        id: "dataMask",
        bands: 1
      }]
  };
}

function evaluatePixel(samples) {
    let index = (samples.B08 - samples.B04) / (samples.B08+samples.B04);
    return {
        ndvi: [index],
        dataMask: [samples.dataMask],
    };
}

"""

In [8]:
evalscript_ndwi = """
//VERSION=3
function setup() {
  return {
    input: [{
      bands: [
        "B03", // Green band
        "B08", // NIR band
        "dataMask" // Mask to exclude the non-valid pixels
      ]
    }],
    output: [
      {
        id: "ndwi",
        bands: 1,
        sampleType: "FLOAT32"
      },
      {
        id: "dataMask",
        bands: 1
      }
    ]
  };
}

function evaluatePixel(samples) {
    let ndwi = (samples.B03 - samples.B08) / (samples.B03 + samples.B08);
    return {
        ndwi: [ndwi],
        dataMask: [samples.dataMask],
    };
}
"""




In [9]:
import json
# Load the GeoJSON file
with open('2020.json', 'r') as f:
    geojson_data = json.load(f)

# Prepare the list for storing Geometry objects for each polygon
geometries = []
average_elevations = []

# Iterate through each feature in the GeoJSON
for feature in geojson_data['features']:
    # Check if the geometry type is MultiPolygon
    if feature['geometry']['type'] == 'MultiPolygon':
        for multi_polygon in feature['geometry']['coordinates']:
            # Each 'multi_polygon' is a list of polygons
            for polygon_coordinates in multi_polygon:
                # Shapely expects the outer ring and inner rings (if any) as separate elements
                if not all(isinstance(coord, list) for coord in polygon_coordinates[0]):
                    polygon_coordinates = [polygon_coordinates]
                # Create a Geometry object for the Polygon
                polygon_geometry = Geometry(geometry={
                    "type": "Polygon",
                    "coordinates": polygon_coordinates
                }, crs=CRS.WGS84)
                geometries.append(polygon_geometry)
                
                # Calculate elevations for the vertices of the polygon
                elevations = []
                for coord in polygon_coordinates[0]:  # Assuming outer ring vertices are enough
                    elevation = get_open_elevation(coord[1], coord[0])  # lat, lon order
                    if elevation is not None:
                        elevations.append(elevation)
                
                # Calculate the average elevation for the polygon
                if elevations:
                    average_elevation = sum(elevations) / len(elevations)
                    average_elevations.append(average_elevation)
                else:
                    average_elevations.append(None)  # In case elevation data couldn't be retrieved

NameError: name 'i' is not defined

In [10]:
print(len(average_elevations))

335


In [17]:
width_in_pixels = round(7360.86 / 200)  # Resulting in a whole number
height_in_pixels = round(16686.9 / 200)  # Resulting in a whole number
combined_df = pd.DataFrame(columns=['geometry_index', 'year', 'ndvi_mean', 'ndwi_mean', 'avg_elevation'])
for i, geometry in enumerate(geometries):
    try:
        request_ndvi = SentinelHubStatistical(
            aggregation=SentinelHubStatistical.aggregation(
                evalscript=evalscript_ndvi,
                time_interval=("2020-01-01T00:00:00Z", "2020-12-30T23:59:59Z"),
                aggregation_interval="P1D",
                size=[200, 200],
    
            ),
            input_data=[
                SentinelHubStatistical.input_data(
                    DataCollection.SENTINEL2_L1C.define_from(
                        name="s2l1c", service_url="https://sh.dataspace.copernicus.eu"
                    ),
                    other_args={"dataFilter": {"maxCloudCoverage": 10}},
                ),
            ],
            geometry=geometry,
            config=config,
        )
        request_ndwi = SentinelHubStatistical(
        aggregation=SentinelHubStatistical.aggregation(
            evalscript=evalscript_ndwi,
            time_interval=("2020-01-01T00:00:00Z", "2020-12-30T23:59:59Z"),
            aggregation_interval="P1D",
            size=[200, 200],
        ),
        input_data=[
            SentinelHubStatistical.input_data(
                DataCollection.SENTINEL2_L1C.define_from(
                    name="s2l1c", service_url="https://sh.dataspace.copernicus.eu"
                ),
                other_args={"dataFilter": {"maxCloudCoverage": 10}},
            ),
        ],
        geometry=geometry,
        config=config,
        )
        response_ndvi = request_ndvi.get_data()
        response_ndwi = request_ndwi.get_data()
        # result_ndvi = read_acquisitions_stats(response_ndvi[0]["data"])
        # result_ndwi = read_acquisitions_stats(response_ndwi[0]["data"])
        # Assume response_ndvi and response_ndwi are now populated with data
        # Convert results to DataFrame
        df_ndvi = pd.DataFrame(read_acquisitions_stats(response_ndvi[0]['data']))
        df_ndwi = pd.DataFrame(read_acquisitions_stats(response_ndwi[0]['data']))
    
        # Convert the 'date' column to datetime format to ease processing
        df_ndvi['date'] = pd.to_datetime(df_ndvi['date'])
        df_ndwi['date'] = pd.to_datetime(df_ndwi['date'])
    
        # Group by year and calculate the mean of the means for each year
        yearly_ndvi = df_ndvi.groupby(df_ndvi['date'].dt.year)['ndvi_mean'].mean().reset_index()
        yearly_ndwi = df_ndwi.groupby(df_ndwi['date'].dt.year)['ndwi_mean'].mean().reset_index()
    
        # Combine the yearly means into a single dataframe
        yearly_combined = pd.merge(yearly_ndvi, yearly_ndwi, on='date', how='outer', suffixes=('_ndvi', '_ndwi'))
        yearly_combined['geometry_index'] = i
        yearly_combined['avg_elevation'] = average_elevations[i]  # Assuming average_elevations list is already populated
    
        # Rename the columns to reflect the data correctly
        yearly_combined.rename(columns={'date': 'year'}, inplace=True)
    
        # Append the yearly combined data to the main combined dataframe
        combined_df = pd.concat([combined_df, yearly_combined], ignore_index=True)
    except:
        continue

# Convert the 'year' column back to a date format if needed
combined_df['year'] = pd.to_datetime(combined_df['year'], format='%Y').dt.to_period('Y')

# Now 'combined_df' contains the combined data for all geometries with the average NDVI, NDWI, and elevation per year
print(combined_df)

/opt/conda/envs/sentinelhub/lib/python3.10/site-packages/sentinelhub/download/sentinelhub_client.py:87: SHRateLimitWarning: Download rate limit hit
  warnings.warn("Download rate limit hit", category=SHRateLimitWarning)
/opt/conda/envs/sentinelhub/lib/python3.10/site-packages/sentinelhub/download/sentinelhub_client.py:87: SHRateLimitWarning: Download rate limit hit
  warnings.warn("Download rate limit hit", category=SHRateLimitWarning)
/opt/conda/envs/sentinelhub/lib/python3.10/site-packages/sentinelhub/download/sentinelhub_client.py:87: SHRateLimitWarning: Download rate limit hit
  warnings.warn("Download rate limit hit", category=SHRateLimitWarning)
/opt/conda/envs/sentinelhub/lib/python3.10/site-packages/sentinelhub/download/sentinelhub_client.py:87: SHRateLimitWarning: Download rate limit hit
  warnings.warn("Download rate limit hit", category=SHRateLimitWarning)
/opt/conda/envs/sentinelhub/lib/python3.10/site-packages/sentinelhub/download/sentinelhub_client.py:87: SHRateLimitWarni

    geometry_index  year  ndvi_mean  ndwi_mean  avg_elevation
0                0  2020   0.076017  -0.051837       0.000000
1                1  2020   0.365846  -0.257702      33.500000
2                2  2020   0.514546  -0.405578     112.666667
3                3  2020   0.341831  -0.279339     102.333333
4                4  2020   0.324602  -0.260452      51.800000
..             ...   ...        ...        ...            ...
321            330  2020  -0.025732   0.189252       0.000000
322            331  2020   0.264045  -0.275291     106.000000
323            332  2020   0.055177  -0.033906      27.666667
324            333  2020   0.185572  -0.185574      19.000000
325            334  2020   0.132292  -0.096265     200.833333

[326 rows x 5 columns]


In [18]:
combined_df.to_csv("combined.csv")

In [ ]:
result_ndvi = read_acquisitions_stats(response_ndvi[0]["data"])
result_ndvi

In [ ]:
result_ndwi = read_acquisitions_stats(response_ndwi[0]["data"])
result_ndwi

In [ ]:
fig_stat, ax_stat = plt.subplots(1, 1, figsize=(12, 6))
t1 = result_ndvi["date"]
ndvi_mean_field1 = result_ndvi["ndvi_mean"]
ndvi_std_field1 = result_ndvi["ndvi_stDev"]
ax_stat.plot(t1, ndvi_mean_field1, label="field 1 mean")
ax_stat.fill_between(
    t1,
    ndvi_mean_field1 - ndvi_std_field1,
    ndvi_mean_field1 + ndvi_std_field1,
    alpha=0.3,
    label="field 1 stDev",
)
ax_stat.tick_params(axis="x", labelrotation=30, labelsize=12)
ax_stat.tick_params(axis="y", labelsize=12)
ax_stat.set_xlabel("Date", size=15)
ax_stat.set_ylabel("NDVI/unitless", size=15)
ax_stat.legend(loc="lower right", prop={"size": 12})
ax_stat.set_title("NDVI time series", fontsize=20)
for label in ax_stat.get_xticklabels()[1::2]:
    label.set_visible(False)

In [ ]:
fig_stat, ax_stat = plt.subplots(1, 1, figsize=(12, 6))
t1 = result_ndwi["date"]
ndwi_mean_field1 = result_ndwi["ndwi_mean"]
ndwi_std_field1 = result_ndwi["ndwi_stDev"]
ax_stat.plot(t1, ndwi_mean_field1, label="field 1 mean")
ax_stat.fill_between(
    t1,
    ndwi_mean_field1 - ndwi_std_field1,
    ndwi_mean_field1 + ndwi_std_field1,
    alpha=0.3,
    label="field 1 stDev",
)
ax_stat.tick_params(axis="x", labelrotation=30, labelsize=12)
ax_stat.tick_params(axis="y", labelsize=12)
ax_stat.set_xlabel("Date", size=15)
ax_stat.set_ylabel("NDWI/unitless", size=15)
ax_stat.legend(loc="lower right", prop={"size": 12})
ax_stat.set_title("NDWI time series", fontsize=20)
for label in ax_stat.get_xticklabels()[1::2]:
    label.set_visible(False)